In [ ]:
import pandas as pd
import numpy as np
import ppscore as pps
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

https://core.ac.uk/download/pdf/55631291.pdf

https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to assess if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]

# Input variables:
### bank client data:
1 - **age** (**numeric**)<br>
2 - **job** : type of job (**categorical**: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - **marital** : marital status (**categorical**: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - **education** (**categorical**: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - **default**: has credit in default? (**categorical**: 'no','yes','unknown')<br>
6 - **housing**: has housing loan? (**categorical**: 'no','yes','unknown')<br>
7 - **loan**: has personal loan? (**categorical**: 'no','yes','unknown')<br>
### related with the last contact of the current campaign:
8 - **contact**: contact communication type (**categorical**: 'cellular','telephone')<br>
9 - **month**: last contact month of year (**categorical**: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - **day_of_week**: last contact day of the week (**categorical**: 'mon','tue','wed','thu','fri')<br>
11 - **duration**: last contact duration, in seconds (**numeric**). **DATA LEAKAGE** Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
### other attributes:
12 - **campaign**: number of contacts performed during this campaign and for this client (**numeric**, includes last contact)<br>
13 - **pdays**: number of days that passed by after the client was last contacted from a previous campaign (**numeric**; 999 means client was not previously contacted)<br>
14 - **previous**: number of contacts performed before this campaign and for this client (**numeric**)<br>
15 - **poutcome**: outcome of the previous marketing campaign (**categorical**: 'failure','nonexistent','success')<br>
### social and economic context attributes
16 - **emp.var.rate**: employment variation rate - quarterly indicator (**numeric**)<br>
17 - **cons.price.idx**: consumer price index - monthly indicator (**numeric**)<br>
18 - **cons.conf.idx**: consumer confidence index - monthly indicator (**numeric**)<br>
19 - **euribor3m**: euribor 3 month rate - daily indicator (**numeric**)<br>
20 - **nr.employed**: number of employees - quarterly indicator (**numeric**)<br>

# Target variable:
21 - **y** - has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:
train = pd.read_csv('train.csv')

# drop 'duration' column due to data leakage and its 'unrealistic' effect on the target variable
train = train.drop(columns='duration')
train.head()

In [ ]:
f, ax = plt.subplots(figsize=(14,4))
ed_order = [
    'unknown', 'illiterate',
    'basic.4y', 'basic.6y',
    'basic.9y', 'high.school',
    'university.degree', 'professional.course'
]
sns.boxenplot(x='education', y='age', hue='y', data=train, order=ed_order, ax=ax)
ax.set_title('With basic.4y education levels older individuals said yes more than younger individuals.');

In [ ]:
f, ax = plt.subplots(figsize=(15,4))

sns.boxenplot(x='job', y='age', hue='y', data=train, ax=ax);
ax.set_title('Older retired individuals, younger students and older housemaids said yes more often.');

In [ ]:
x = train.groupby(['job', 'marital', 'education', 'housing', 'loan', 'poutcome', 'y']).count().loc[:, ['age']]
x = x.sort_index()
x

In [ ]:
x.age.max()

In [ ]:
x.loc[x.age==x.age.max(),:]

In [ ]:
x.loc[('admin.', 'married', 'university.degree', 'yes', 'no', 'nonexistent',),:]

In [ ]:
x.loc[('admin.', 'married', 'university.degree', 'yes', 'no'),:]

In [ ]:
train.query("job=='admin.' and marital=='married' and education=='university.degree' and housing=='yes' and loan=='no'")

In [ ]:
import plotly.express as px

In [ ]:
train.head()

In [ ]:
# # PARALLEL CATEGORIES - THREE CATEGORIES AND WIN %
# box_df = box_df.assign(win_num=box_df.win.astype(int))
# travel_cats = ['home', 'travelled']
# fig = px.parallel_categories(box_df, dimensions=travel_cats, color='win_num')

In [ ]:
g = train.copy()
g.y = g.y.apply(lambda x: 1 if x == 'yes' else 0)
f = px.parallel_categories(g, dimensions=['y', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'poutcome'], color='y')
f.show()

In [ ]:
train.loc[:, ['poutcome', 'previous', 'campaign']]

In [ ]:
train['cur_prev_ratio'] = (train.campaign/train.previous).replace([np.inf], 0)

In [ ]:
train.cur_prev_ratio = train.cur_prev_ratio.replace(to_replace=0, train.cur_prev_ratio.max())

In [ ]:
# f = px.treemap(train.loc[train.poutcome!='nonexistent',:],
#                path=['y', 'poutcome', 'job'],
#                width=1600, height=750,
#                color_continuous_scale='RdBu',
#                color='previous'
#               )
# f.show()

In [ ]:
f = px.treemap(train,
               path=['poutcome', 'job', 'y'],
               width=1600, height=750,
               color_continuous_scale='RdBu',
               color='cur_prev_ratio'
              )
f.show()

In [ ]:
f = px.treemap(train.loc[train.poutcome!='nonexistent',:],
               path=['poutcome', 'job', 'y'],
               width=1600, height=750,
               color_continuous_scale='RdBu',
               color='previous'
              )
f.show()

In [ ]:
# f = px.treemap(train.loc[train.poutcome=='nonexistent',:],
#                path=['y', 'job'],
#                width=1200, height=750
#               )
# f.show()

In [ ]:
jobs = train.groupby(['job', 'y'])

In [ ]:
jobs[['marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']].agg(lambda x:x.value_counts().index[0])

In [ ]:
jobs.describe().loc[:, 'age']

In [ ]:
jobs.agg(['mean', 'median', 'std'])